##**Mounting Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Importing Libraries**

In [2]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.1 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

#Hyperparameter Optimization
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-3-ff41bca45932>:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


##**Determining File Path and Separating into Subfolders**

In [4]:
data_dir = '/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/'

data_path = "/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/"
cataract_path = os.path.join(data_path, "Cataract")
normal_path = os.path.join(data_path, "Normal")

print(os.listdir("/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/"))

['Glaucoma', 'Normal', 'Cataract', 'Diabetic Retinopathy']


##**Specifying Image Size**

In [5]:
img_height, img_width = 128, 128

##**Loading Images**

In [6]:
def load_images(folder_path, label):
    images = []
    labels = []
    for idx, filename in enumerate(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error: Unable to read image {img_path}")
            continue
        img = cv2.resize(img, (img_height, img_width))
        images.append(img)
        labels.append(label)
        print(f"Image {idx + 1} loaded from {img_path}")
    return np.array(images), np.array(labels)

cataract_images, cataract_labels = load_images(cataract_path, 0)
normal_images, normal_labels = load_images(normal_path, 1)

Image 1 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/1167_right.jpg
Image 2 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/1115_left.jpg
Image 3 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2074_right.jpg
Image 4 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2163_right.jpg
Image 5 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/1468_left.jpg
Image 6 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2106_right.jpg
Image 7 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2101_left.jpg
Image 8 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/103_left.jpg
Image 9 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2151_left.jpg
Image 10 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2126_right.jpg
Image

##**Concatenating the Data Set as Cataract and Normal**

In [7]:
x = np.concatenate((cataract_images, normal_images), axis=0)
y = np.concatenate((cataract_labels, normal_labels), axis=0)

###Displaying the Number of Data Samples for X and Y

In [8]:
print("Number of data samples:", x.shape[0])
print("Number of data samples:", y.shape[0])

Number of data samples: 2112
Number of data samples: 2112


###Splitting the Data Set into Training (80%) and Test (20%) Sets

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 38)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 38)

###Training and Finalizing the CNN Model

In [10]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [11]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [12]:
def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv1', min_value=0.2, max_value=0.5, step=0.1)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 3
    model.add(Dense(units=hp.Int('dense3_units', min_value=64, max_value=256, step=16)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [13]:
# Creating the tuner using RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Maximum number of models to try
    directory='my_dir',  # Directory to store saved models and results
    project_name='retinal_imaging_hyperparameter_tuning'  # Name of the tuning project
)

# Fitting the tuner with training data
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# Getting the best hyperparameters and building the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Training the best model on the training set
best_model.fit(x_train, train_yCl, epochs=15, validation_data=(x_val, valid_yCl))

Trial 5 Complete [00h 00m 30s]
val_accuracy: 0.8757396340370178

Best val_accuracy So Far: 0.8875739574432373
Total elapsed time: 00h 05m 43s
Epoch 1/15
48/48 [==============================] - 6s 96ms/step - loss: 75.8840 - accuracy: 0.6283 - val_loss: 0.5875 - val_accuracy: 0.5207
Epoch 2/15
48/48 [==============================] - 4s 93ms/step - loss: 0.5151 - accuracy: 0.7961 - val_loss: 0.5294 - val_accuracy: 0.7515
Epoch 3/15
48/48 [==============================] - 4s 94ms/step - loss: 0.4434 - accuracy: 0.8309 - val_loss: 0.4040 - val_accuracy: 0.8462
Epoch 4/15
48/48 [==============================] - 4s 92ms/step - loss: 0.3697 - accuracy: 0.8526 - val_loss: 0.3437 - val_accuracy: 0.8698
Epoch 5/15
48/48 [==============================] - 5s 94ms/step - loss: 0.3344 - accuracy: 0.8645 - val_loss: 0.3380 - val_accuracy: 0.8639
Epoch 6/15
48/48 [==============================] - 5s 97ms/step - loss: 0.3410 - accuracy: 0.8651 - val_loss: 0.3229 - val_accuracy: 0.8639
Epoch 7/15


In [14]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

6/6 [==============================] - 0s 19ms/step - loss: 0.3206 - accuracy: 0.8639
Validation Accuracy:  0.8639053106307983
14/14 [==============================] - 1s 45ms/step - loss: 0.3932 - accuracy: 0.8487
Test Accuracy:  0.8486997485160828
48/48 [==============================] - 1s 15ms/step - loss: 0.2562 - accuracy: 0.8842
Training Accuracy:  0.8842105269432068


In [15]:
# Making predictions on the test data
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Calculating metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))

# Printing the results
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Displaying the Confusion Matrix
print('Confusion Matrix:')
print(conf_matrix)

# Showing the Classification Report
print('Classification Report:')
print(class_report)

14/14 [==============================] - 0s 14ms/step
Test Accuracy: 0.8487
Precision: 0.8753
Recall: 0.8487
F1 Score: 0.8444
Confusion Matrix:
[[137  61]
 [  3 222]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.69      0.81       198
           1       0.78      0.99      0.87       225

    accuracy                           0.85       423
   macro avg       0.88      0.84      0.84       423
weighted avg       0.88      0.85      0.84       423



###Splitting the Data Set into Training (65%) and Test (35%) Sets

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.35, random_state = 38)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 38)

###Training and Finalizing the CNN Model

In [17]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [18]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [19]:
def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv1', min_value=0.2, max_value=0.5, step=0.1)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 3
    model.add(Dense(units=hp.Int('dense3_units', min_value=64, max_value=256, step=16)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [20]:
# Creating the tuner using RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Maximum number of models to try
    directory='my_dir',  # Directory to store saved models and results
    project_name='retinal_imaging_hyperparameter_tuning'  # Name of the tuning project
)

# Fitting the tuner with training data
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# Getting the best hyperparameters and building the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Training the best model on the training set
best_model.fit(x_train, train_yCl, epochs=15, validation_data=(x_val, valid_yCl))

Reloading Tuner from my_dir/retinal_imaging_hyperparameter_tuning/tuner0.json
Epoch 1/15
39/39 [==============================] - 8s 146ms/step - loss: 108.9585 - accuracy: 0.5762 - val_loss: 0.6701 - val_accuracy: 0.8261
Epoch 2/15
39/39 [==============================] - 4s 94ms/step - loss: 0.5830 - accuracy: 0.7472 - val_loss: 0.6934 - val_accuracy: 0.5072
Epoch 3/15
39/39 [==============================] - 4s 105ms/step - loss: 0.4603 - accuracy: 0.8177 - val_loss: 0.5063 - val_accuracy: 0.7391
Epoch 4/15
39/39 [==============================] - 4s 94ms/step - loss: 0.4047 - accuracy: 0.8412 - val_loss: 0.5167 - val_accuracy: 0.6812
Epoch 5/15
39/39 [==============================] - 4s 95ms/step - loss: 0.3676 - accuracy: 0.8558 - val_loss: 0.5547 - val_accuracy: 0.7101
Epoch 6/15
39/39 [==============================] - 4s 97ms/step - loss: 0.3481 - accuracy: 0.8582 - val_loss: 0.3648 - val_accuracy: 0.8333
Epoch 7/15
39/39 [==============================] - 4s 92ms/step - loss:

In [21]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

5/5 [==============================] - 0s 14ms/step - loss: 0.3510 - accuracy: 0.8623
Validation Accuracy:  0.8623188138008118
24/24 [==============================] - 1s 26ms/step - loss: 1.1822 - accuracy: 0.8649
Test Accuracy:  0.8648648858070374
39/39 [==============================] - 1s 15ms/step - loss: 0.2768 - accuracy: 0.8857
Training Accuracy:  0.885737419128418


In [22]:
# Making predictions on the test data
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Calculating metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))

# Printing the results
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Displaying the Confusion Matrix
print('Confusion Matrix:')
print(conf_matrix)

# Showing the Classification Report
print('Classification Report:')
print(class_report)

24/24 [==============================] - 0s 13ms/step
Test Accuracy: 0.8649
Precision: 0.8761
Recall: 0.8649
F1 Score: 0.8627
Confusion Matrix:
[[258  84]
 [ 16 382]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.75      0.84       342
           1       0.82      0.96      0.88       398

    accuracy                           0.86       740
   macro avg       0.88      0.86      0.86       740
weighted avg       0.88      0.86      0.86       740

